# Bagging 분류

개별적으로 수행된 알고리즘 중에서 최적의 알고리즘에 대해 샘플을 여러 번 뽑아(Bootstrap) 각 모델을 학습시켜 결과물을 집계(Aggregation)하는 방법.

우선적으로 주어진 데이터에 적합한 최적의 모델을 선정해야 한다.

Votting의 결과도 Bagging의 대상이 될 수 있으며 Bagging의 결과도 Votting의 대상이 될 수 있지만 머신 성능의 문제로 잘 사용되지는 않는다.

## #01. 준비작업

### [1] 패키지 가져오기

In [1]:
# 연결된 모듈이 업데이트 되면 즉시 자동 로드함
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings(action='ignore')

from hossam.util import *
from hossam.plot import *
from hossam.analysis import *
from hossam.classification import *

# 배깅 앙상블
from sklearn.ensemble import BaggingClassifier

### [2] 데이터 가져오기 + 전처리

`0`값을 포함해서는 안되는 변수에 대해 평균으로 대체

In [ ]:
origin = my_read_excel("https://data.hossam.kr/mldata/pima_indians_diabetes.xlsx", categories=['Outcome'], info=False)

# 정제 대상 필드명
zero_features = ['Glucose', 'BloodPressure','SkinThickness','Insulin','BMI']

# 0값을 결측치로 대체후 평균으로 다시 치환
df = origin.copy()
df[zero_features] = df[zero_features].replace(0, np.nan)
df2 = my_replace_missing_value(df)

# 결과확인
total_count = len(origin)

for feature in zero_features:
    zero_count = origin[origin[feature] == 0][feature].count()
    print('{0} 0 건수는 {1}, 퍼센트는 {2:.2f} %'.format(feature, zero_count, 100*zero_count/total_count))

print("---" * 30)

for feature in zero_features:
    zero_count = df2[df2[feature] == 0][feature].count()
    print('{0} 0 건수는 {1}, 퍼센트는 {2:.2f} %'.format(feature, zero_count, 100*zero_count/total_count))

### [3] 훈련/검증 데이터 분할 및 데이터 표준화

In [ ]:
x_train, x_test, y_train, y_test = my_train_test_split(df2, 'Outcome', scalling=True)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

## #02. 기본 Bagging

### [1] 이 데이터에 대한 최적 모형 찾기

In [ ]:
best_estimator = my_classification(x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test, pruning=False)
best_estimator

In [ ]:
best_estimator['best']

### [3] Bagging 모형 구현

In [ ]:
bagging = BaggingClassifier(estimator=best_estimator['best'], 
                            n_estimators=10, # 부트스트랩 샘플 개수
                            bootstrap=True,  # 복원 추출,  False이면 비복원 추출
                            random_state=get_random_state(),
                            
                            # 하나의 예측기에 들어가는 샘플에 대하여 컬럼의 중복 사용여부를 결정
                            bootstrap_features=False, 
                            n_jobs=get_n_jobs())

bagging.fit(x_train, y_train)

my_classification_result(estimator=bagging, x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test)